In [1]:
# !pip install textblob

In [4]:
from textblob import TextBlob
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

dataset = pd.read_csv("finalData.csv")
print(dataset.count())
# dataset = dataset[['description']]

Unnamed: 0     32120
index          32120
description    32120
category       17120
civic_issue    32120
dtype: int64


In [7]:
def sentiment_analysis(dataset):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
# Create a function to get the polarity
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity

    def getAnalysis(score):
        if score < 0.3:
            return 'Complaint'
        elif score > 0.3 and score < 0.4:
            return 'Neutral Feedback'
        else:
            return 'Compliment'

    dataset['Subjectivity'] =    dataset.description.apply(getSubjectivity)
    dataset['Polarity'] = dataset.description.apply(getPolarity)
    dataset['Sentiment'] = dataset['Polarity'].apply(getAnalysis )
    return dataset

df = sentiment_analysis(dataset)
#  if score < 0:
#   return ‘Negative’
#  elif score == 0:
#   return ‘Neutral’
#  else:
#   return ‘Positive’

In [8]:
df.head(10)

,Unnamed: 0,index,description,category,civic_issue,Subjectivity,Polarity,Sentiment
0,0,1,Garbage behind the temple,Waste/Garbage,1,0.700000,-0.400000,Complaint
1,1,2,Air pollution,Air,1,0.000000,0.000000,Complaint
2,2,3,Air pollution in hebbal,Air,1,0.000000,0.000000,Complaint
3,3,4,Garbage is dumped near BES,Waste/Garbage,1,0.400000,0.100000,Complaint
4,4,5,Leaf and garbage burning on Shakthi Ganapathi ...,Air,1,0.100000,-0.050000,Complaint
5,5,6,There is a lot of garbage dump in and around t...,Waste/Garbage,1,0.400000,-0.250000,Complaint
6,6,7,Replace broken streetlight. Complaint #5 about...,Streetlights,1,0.241667,-0.233333,Complaint
7,7,8,Red Matiz parked on the footpath for more than...,Traffic/Parking,1,0.200000,0.150000,Complaint
8,8,9,Cars have been using the footpath as a parking...,Traffic/Parking,1,0.350000,-0.125000,Complaint
9,9,10,Congestion everyday and at all times due to im...,Traffic/Parking,1,0.487500,-0.162500,Complaint
